In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query

from datetime import datetime

종목코드열람

In [2]:
shcode_list = query.query_shcode_list_today()

In [3]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [4]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [5]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 500
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 5:
            average_volume_test[ind] = 1
            
    return average_volume_test

#최종

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')

In [6]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.05
volume_money = 20000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v3' +'.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-11-19_15:46:42 : , 0 , 000020 , 동화약품 -------------------  
O_1. ratio_test 통과 0 , 000020 , 동화약품
X_2. total_volume_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-11-19_15:46:44 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-11-19_15:46:44 : , 2 , 000050 , 경방 -------------------  
X_1. ratio_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-11-19_15:46:45 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-11-19_15:46:45 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-11-19_15:46:46 : , 5 , 000075 , 삼양홀딩스우 -------------------  
O_1. ratio_test 통과 5 , 000075 , 삼양홀딩스우
X_2. total_volume_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-11-19_15:46:46 : , 6 , 000080 , 하이트진로 -------------------  
O_1. ratio_test 통과 6 , 000080 , 하이트진로

O_1. ratio_test 통과 42 , 000660 , SK하이닉스
O_2. total_volume_test 통과 42 , 000660 , SK하이닉스
X_3. average_volume_test 없음 42 , 000660 , SK하이닉스
------------------- test 시작_2019-11-19_15:47:4 : , 43 , 000670 , 영풍 -------------------  
O_1. ratio_test 통과 43 , 000670 , 영풍
O_2. total_volume_test 통과 43 , 000670 , 영풍
X_3. average_volume_test 없음 43 , 000670 , 영풍
------------------- test 시작_2019-11-19_15:47:5 : , 44 , 000680 , LS네트웍스 -------------------  
O_1. ratio_test 통과 44 , 000680 , LS네트웍스
X_2. total_volume_test 없음 44 , 000680 , LS네트웍스
------------------- test 시작_2019-11-19_15:47:5 : , 45 , 000700 , 유수홀딩스 -------------------  
O_1. ratio_test 통과 45 , 000700 , 유수홀딩스
X_2. total_volume_test 없음 45 , 000700 , 유수홀딩스
------------------- test 시작_2019-11-19_15:47:6 : , 46 , 000720 , 현대건설 -------------------  
O_1. ratio_test 통과 46 , 000720 , 현대건설
O_2. total_volume_test 통과 46 , 000720 , 현대건설
X_3. average_volume_test 없음 46 , 000720 , 현대건설
------------------- test 시작_2019-11-19_15:47:6 : , 47 , 000725 , 현대건설

O_1. ratio_test 통과 84 , 001380 , SG충방
O_2. total_volume_test 통과 84 , 001380 , SG충방
O_3. average_volume_test 통과 84 , 001380 , SG충방
------------------- test 완료 : , 84 , 001380 , SG충방 ------------------- 
------------------- test 시작_2019-11-19_15:47:24 : , 85 , 001390 , KG케미칼 -------------------  
O_1. ratio_test 통과 85 , 001390 , KG케미칼
X_2. total_volume_test 없음 85 , 001390 , KG케미칼
------------------- test 시작_2019-11-19_15:47:25 : , 86 , 001420 , 태원물산 -------------------  
O_1. ratio_test 통과 86 , 001420 , 태원물산
X_2. total_volume_test 없음 86 , 001420 , 태원물산
------------------- test 시작_2019-11-19_15:47:25 : , 87 , 001430 , 세아베스틸 -------------------  
O_1. ratio_test 통과 87 , 001430 , 세아베스틸
X_2. total_volume_test 없음 87 , 001430 , 세아베스틸
------------------- test 시작_2019-11-19_15:47:26 : , 88 , 001440 , 대한전선 -------------------  
O_1. ratio_test 통과 88 , 001440 , 대한전선
O_2. total_volume_test 통과 88 , 001440 , 대한전선
O_3. average_volume_test 통과 88 , 001440 , 대한전선
------------------- test 완료 : , 88 , 0014

O_1. ratio_test 통과 126 , 002070 , 남영비비안
O_2. total_volume_test 통과 126 , 002070 , 남영비비안
O_3. average_volume_test 통과 126 , 002070 , 남영비비안
------------------- test 완료 : , 126 , 002070 , 남영비비안 ------------------- 
------------------- test 시작_2019-11-19_15:47:44 : , 127 , 002100 , 경농 -------------------  
O_1. ratio_test 통과 127 , 002100 , 경농
O_2. total_volume_test 통과 127 , 002100 , 경농
O_3. average_volume_test 통과 127 , 002100 , 경농
------------------- test 완료 : , 127 , 002100 , 경농 ------------------- 
------------------- test 시작_2019-11-19_15:47:45 : , 128 , 002140 , 고려산업 -------------------  
O_1. ratio_test 통과 128 , 002140 , 고려산업
O_2. total_volume_test 통과 128 , 002140 , 고려산업
O_3. average_volume_test 통과 128 , 002140 , 고려산업
------------------- test 완료 : , 128 , 002140 , 고려산업 ------------------- 
------------------- test 시작_2019-11-19_15:47:45 : , 129 , 002150 , 도화엔지니어링 -------------------  
O_1. ratio_test 통과 129 , 002150 , 도화엔지니어링
O_2. total_volume_test 통과 129 , 002150 , 도화엔지니어링
O_3. average

O_1. ratio_test 통과 163 , 002840 , 미원상사
X_2. total_volume_test 없음 163 , 002840 , 미원상사
------------------- test 시작_2019-11-19_15:48:1 : , 164 , 002870 , 신풍제지 -------------------  
O_1. ratio_test 통과 164 , 002870 , 신풍제지
X_2. total_volume_test 없음 164 , 002870 , 신풍제지
------------------- test 시작_2019-11-19_15:48:2 : , 165 , 002880 , 대유에이텍 -------------------  
O_1. ratio_test 통과 165 , 002880 , 대유에이텍
O_2. total_volume_test 통과 165 , 002880 , 대유에이텍
X_3. average_volume_test 없음 165 , 002880 , 대유에이텍
------------------- test 시작_2019-11-19_15:48:2 : , 166 , 002900 , 동양물산 -------------------  
O_1. ratio_test 통과 166 , 002900 , 동양물산
O_2. total_volume_test 통과 166 , 002900 , 동양물산
O_3. average_volume_test 통과 166 , 002900 , 동양물산
------------------- test 완료 : , 166 , 002900 , 동양물산 ------------------- 
------------------- test 시작_2019-11-19_15:48:3 : , 167 , 002920 , 유성기업 -------------------  
O_1. ratio_test 통과 167 , 002920 , 유성기업
X_2. total_volume_test 없음 167 , 002920 , 유성기업
------------------- test 시작_20

...

O_1. ratio_test 통과 2754 , 251970 , 펌텍코리아
O_2. total_volume_test 통과 2754 , 251970 , 펌텍코리아
X_3. average_volume_test 없음 2754 , 251970 , 펌텍코리아
------------------- test 시작_2019-11-19_16:8:57 : , 2755 , 252500 , 세화피앤씨 -------------------  
O_1. ratio_test 통과 2755 , 252500 , 세화피앤씨
O_2. total_volume_test 통과 2755 , 252500 , 세화피앤씨
O_3. average_volume_test 통과 2755 , 252500 , 세화피앤씨
------------------- test 완료 : , 2755 , 252500 , 세화피앤씨 ------------------- 
------------------- test 시작_2019-11-19_16:8:58 : , 2756 , 253450 , 스튜디오드래곤 -------------------  
O_1. ratio_test 통과 2756 , 253450 , 스튜디오드래곤
O_2. total_volume_test 통과 2756 , 253450 , 스튜디오드래곤
X_3. average_volume_test 없음 2756 , 253450 , 스튜디오드래곤
------------------- test 시작_2019-11-19_16:8:58 : , 2757 , 253590 , 네오셈 -------------------  
O_1. ratio_test 통과 2757 , 253590 , 네오셈
O_2. total_volume_test 통과 2757 , 253590 , 네오셈
O_3. average_volume_test 통과 2757 , 253590 , 네오셈
------------------- test 완료 : , 2757 , 253590 , 네오셈 ------------------- 
-----------

O_1. ratio_test 통과 2788 , 263920 , 블러썸엠앤씨
X_2. total_volume_test 없음 2788 , 263920 , 블러썸엠앤씨
------------------- test 시작_2019-11-19_16:9:13 : , 2789 , 264290 , 한화에이스스팩3호 -------------------  
O_1. ratio_test 통과 2789 , 264290 , 한화에이스스팩3호
O_2. total_volume_test 통과 2789 , 264290 , 한화에이스스팩3호
O_3. average_volume_test 통과 2789 , 264290 , 한화에이스스팩3호
------------------- test 완료 : , 2789 , 264290 , 한화에이스스팩3호 ------------------- 
------------------- test 시작_2019-11-19_16:9:13 : , 2790 , 264450 , 유비쿼스 -------------------  
O_1. ratio_test 통과 2790 , 264450 , 유비쿼스
O_2. total_volume_test 통과 2790 , 264450 , 유비쿼스
O_3. average_volume_test 통과 2790 , 264450 , 유비쿼스
------------------- test 완료 : , 2790 , 264450 , 유비쿼스 ------------------- 
------------------- test 시작_2019-11-19_16:9:13 : , 2791 , 264660 , 씨앤지하이테크 -------------------  
O_1. ratio_test 통과 2791 , 264660 , 씨앤지하이테크
O_2. total_volume_test 통과 2791 , 264660 , 씨앤지하이테크
O_3. average_volume_test 통과 2791 , 264660 , 씨앤지하이테크
------------------- test 완료 : , 27

O_1. ratio_test 통과 2825 , 290270 , 휴네시온
O_2. total_volume_test 통과 2825 , 290270 , 휴네시온
O_3. average_volume_test 통과 2825 , 290270 , 휴네시온
------------------- test 완료 : , 2825 , 290270 , 휴네시온 ------------------- 
------------------- test 시작_2019-11-19_16:9:30 : , 2826 , 290380 , 대유 -------------------  
O_1. ratio_test 통과 2826 , 290380 , 대유
O_2. total_volume_test 통과 2826 , 290380 , 대유
O_3. average_volume_test 통과 2826 , 290380 , 대유
------------------- test 완료 : , 2826 , 290380 , 대유 ------------------- 
------------------- test 시작_2019-11-19_16:9:30 : , 2827 , 290550 , 디케이티 -------------------  
O_1. ratio_test 통과 2827 , 290550 , 디케이티
O_2. total_volume_test 통과 2827 , 290550 , 디케이티
X_3. average_volume_test 없음 2827 , 290550 , 디케이티
------------------- test 시작_2019-11-19_16:9:30 : , 2828 , 290650 , 엘앤씨바이오 -------------------  
O_1. ratio_test 통과 2828 , 290650 , 엘앤씨바이오
O_2. total_volume_test 통과 2828 , 290650 , 엘앤씨바이오
O_3. average_volume_test 통과 2828 , 290650 , 엘앤씨바이오
------------------- test 완료 

X_1. ratio_test 없음 2865 , 317030 , 케이비17호스팩
------------------- test 시작_2019-11-19_16:9:48 : , 2866 , 317120 , 라닉스 -------------------  
O_1. ratio_test 통과 2866 , 317120 , 라닉스
O_2. total_volume_test 통과 2866 , 317120 , 라닉스
X_3. average_volume_test 없음 2866 , 317120 , 라닉스
------------------- test 시작_2019-11-19_16:9:48 : , 2867 , 317240 , 하이제4호스팩 -------------------  
X_1. ratio_test 없음 2867 , 317240 , 하이제4호스팩
------------------- test 시작_2019-11-19_16:9:49 : , 2868 , 317320 , 한화에스비아이스팩 -------------------  
O_1. ratio_test 통과 2868 , 317320 , 한화에스비아이스팩
O_2. total_volume_test 통과 2868 , 317320 , 한화에스비아이스팩
O_3. average_volume_test 통과 2868 , 317320 , 한화에스비아이스팩
------------------- test 완료 : , 2868 , 317320 , 한화에스비아이스팩 ------------------- 
------------------- test 시작_2019-11-19_16:9:49 : , 2869 , 317330 , 덕산테코피아 -------------------  
O_1. ratio_test 통과 2869 , 317330 , 덕산테코피아
O_2. total_volume_test 통과 2869 , 317330 , 덕산테코피아
X_3. average_volume_test 없음 2869 , 317330 , 덕산테코피아
------------------- tes

O_1. ratio_test 통과 2899 , 950110 , SBI핀테크솔루션즈
X_2. total_volume_test 없음 2899 , 950110 , SBI핀테크솔루션즈
------------------- test 시작_2019-11-19_16:10:3 : , 2900 , 950130 , 엑세스바이오 -------------------  
O_1. ratio_test 통과 2900 , 950130 , 엑세스바이오
O_2. total_volume_test 통과 2900 , 950130 , 엑세스바이오
O_3. average_volume_test 통과 2900 , 950130 , 엑세스바이오
------------------- test 완료 : , 2900 , 950130 , 엑세스바이오 ------------------- 
------------------- test 시작_2019-11-19_16:10:3 : , 2901 , 950140 , 잉글우드랩 -------------------  
O_1. ratio_test 통과 2901 , 950140 , 잉글우드랩
X_2. total_volume_test 없음 2901 , 950140 , 잉글우드랩
------------------- test 시작_2019-11-19_16:10:4 : , 2902 , 950160 , 코오롱티슈진 -------------------  
O_1. ratio_test 통과 2902 , 950160 , 코오롱티슈진
O_2. total_volume_test 통과 2902 , 950160 , 코오롱티슈진
O_3. average_volume_test 통과 2902 , 950160 , 코오롱티슈진
------------------- test 완료 : , 2902 , 950160 , 코오롱티슈진 ------------------- 
------------------- test 시작_2019-11-19_16:10:4 : , 2903 , 950170 , JTC ------------------